In [2]:
import os
import sys
import pickle
import numpy as np
import keras_tuner as kt
from tensorflow import keras

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

2025-01-29 07:52:02.840056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
with open("../../pickles/X_train_aug.pkl", "rb") as file:
    X_train = pickle.load(file)

with open("../../pickles/y_train_aug.pkl", "rb") as file:
    y_train = pickle.load(file)

with open("../../pickles/labels.pkl", "rb") as file:
    labels = pickle.load(file)

# with open("../../pickles/resized_images.pkl", "rb") as file:
#     resized = pickle.load(file)




In [4]:
print(labels)
print(labels.count) 
print(X_train[0])
print(y_train[0])
print(X_train.shape)
print(y_train.shape)    
print(X_train.shape[1:]) 

0       forest
1       forest
2       forest
3       forest
4       forest
         ...  
2779       sea
2780       sea
2781       sea
2782       sea
2783       sea
Name: label, Length: 17034, dtype: object
<bound method Series.count of 0       forest
1       forest
2       forest
3       forest
4       forest
         ...  
2779       sea
2780       sea
2781       sea
2782       sea
2783       sea
Name: label, Length: 17034, dtype: object>
[[[0.16078432 0.22352941 0.11764706]
  [0.08235294 0.14509805 0.04313726]
  [0.2627451  0.30588236 0.24313726]
  ...
  [0.21568628 0.4509804  0.6862745 ]
  [0.40784314 0.6039216  0.8117647 ]
  [0.4627451  0.6313726  0.83137256]]

 [[0.39215687 0.44313726 0.34117648]
  [0.02745098 0.07450981 0.        ]
  [0.03137255 0.0627451  0.00392157]
  ...
  [0.07058824 0.30980393 0.5568628 ]
  [0.2509804  0.45490196 0.6901961 ]
  [0.32941177 0.52156866 0.7490196 ]]

 [[0.06666667 0.09019608 0.        ]
  [0.17254902 0.19215687 0.10588235]
  [0.02745098 0.04313

In [5]:
print(np.unique(y_train))  # Ensure this matches expected labels (0 to 5)
count = np.unique(y_train, return_counts=True)
print(count) 

[0. 1.]
(array([0., 1.]), array([136270,  27254]))


In [ ]:
# y_encoder = LabelEncoder().fit(y_train)
# y = y_encoder.transform(y_train)

# # Convert values to numpy arrays
# X = np.array(X_train)

# Split the training dataset into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
#X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


In [ ]:
def build_cnn_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters=hp.Int('filters', min_value=32, max_value=128, step=32),
            kernel_size=hp.Choice('kernel_size', values=[3, 5]),
            activation='relu',
            input_shape=(150, 150, 3)  # Modify for your dataset
        ),
        keras.layers.MaxPooling2D(pool_size=(2, 2)),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('units', min_value=32, max_value=128, step=32),
            activation='relu'
        ),
        keras.layers.Dense(6, activation='softmax')  
    ])

    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

# Initialize the RandomSearch Tuner
tuner = kt.RandomSearch(
    hypermodel=build_cnn_model,
    objective='val_accuracy',
    max_trials=10,  # Number of hyperparameter sets to try
    executions_per_trial=1,  # Number of times to train each model
    directory='hyperparameter_tuning',
    project_name='cnn_randomsearch'
)


# Start hyperparameter tuning
tuner.search(X_train, y_train, validation_data=(X_train, y_train), epochs=10, batch_size=32)

# Get the best hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best hyperparameters:", best_hps.values)

# Retrieve and build the best model
model = tuner.hypermodel.build(best_hps)
model.summary()

Reloading Tuner from hyperparameter_tuning/cnn_randomsearch/tuner0.json

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
64                |32                |filters
5                 |3                 |kernel_size
128               |64                |units
0.00046926        |0.0028135         |learning_rate

Epoch 1/10
  1/682 ━━━━━━━━━━━━━━━━━━━━ 39:22 3s/step - accuracy: 0.2500 - loss: 1.8313

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])



In [ ]:
# Train the model
batch_size = 32
epochs = 10
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=epochs
)

In [ ]:
model.save("../../models/keras_model.h5")

In [ ]:
img_path = "../../model/pred/93.jpg"  # Change this to your image path
img_path = "../../resources/buildings/66.jpg"  # Change this to your image path

img = image.load_img(img_path, target_size=(150, 150))  # Resize to match model input size
img.show()
img_array = image.img_to_array(img)
img_array = img_array / 255.0  # Normalize if the model was trained with normalized images

# Expand dimensions to match the batch shape (model expects a batch, not a single image)
img_array = np.expand_dims(img_array, axis=0)
print(img_array.shape)
print(img_array.dtype)
# Make prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)
# Print predictions
text = predictions[0][predicted_class]
print(predictions)
print(predicted_class)

In [ ]:
predicted_class = np.argmax(predictions, axis=1)  # Get class index
print("Predicted Class:", predicted_class)